# Live Capturing

In [1]:
#live video capture for 5 seconds
import cv2
import dlib
import time

def capture_live_lip_movements(output_path, fps=29, duration=3):
    """
    Captures a live video of lip movements for a specified duration and saves it.

    Args:
        output_path: The path and filename to save the output video.
        fps: The desired frame rate for the output video (default: 29).
        duration: The duration in seconds to capture the video (default: 3).
    """

    # Load face detector and facial landmarks predictor from dlib
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\shape_predictor_68_face_landmarks.dat")

    # Open video capture object (0 for default webcam)
    cap = cv2.VideoCapture(0)

    # Get video frame dimensions
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    

    # Create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    start_time = time.time()  # Capture start time

    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()

        # Break the loop if video capture fails
        if not ret:
            break

        # Check if capture duration has been reached
        elapsed_time = time.time() - start_time
        if elapsed_time >= duration:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = detector(gray_frame)

        # Process only if a face is detected
        if len(faces) > 0:
            face = faces[0]  # Assuming only one face is present

            # Detect facial landmarks
            shape = predictor(gray_frame, face)

            # Extract lip landmarks (indices based on dlib's model)
            lip_landmarks = [(shape.part(i).x, shape.part(i).y) for i in range(48, 58)]

            # You can add additional processing or visualization of the lip landmarks here

        # Write the frame to the output video
        out.write(frame)

        # Display the frame
        cv2.imshow('Live Lip Movements', frame)

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and writer objects
    cap.release()
    out.release()

    # Destroy all OpenCV windows
    cv2.destroyAllWindows()

# Example usage
output_path = r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\outputs\Extractions\Demo\video_captured10.mp4"
capture_live_lip_movements(output_path)


# Lip Extraction 

In [1]:
import cv2
import dlib
import numpy as np

# Load face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\shape_predictor_68_face_landmarks.dat")

# Define video file path (replace with your video path)
video_path = r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\outputs\Extractions\Demo\video_captured10.mp4"

# Open video capture
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file:", video_path)
    exit()

# Get video properties (optional, adjust output FPS if needed)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# --- Optional: Define video output properties for lower face region ---
# Adjust codec (fourcc) and filename as needed
output_path = r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\outputs\Extractions\Demo\test.mp4"
output_fps = 29
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, output_fps, (112, 80))

while True:
    # Capture frame
    ret, frame = cap.read()

    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)

    for face in faces:
        # Predict facial landmarks
        landmarks = predictor(gray, face)

        # Extract lower face landmarks (below the nose until the jaw)
        lower_face_landmarks = landmarks.parts()[48:58]

        # Convert lower face landmarks to NumPy arrays
        lower_face_points = np.array([(p.x, p.y) for p in lower_face_landmarks], dtype=np.int32)

        # Get the bounding box for the lower part of the face
        x, y, w, h = cv2.boundingRect(lower_face_points)

        # Crop the lower part of the face
        lower_face = frame[y:y+h, x:x+w]

        # Resize the lower face to the desired frame size (112x80)
        lower_face = cv2.resize(lower_face, (112, 80))

        # Write the extracted lower face region to video (optional)
        out.write(lower_face)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Release the VideoWriter object (if used)
cv2.destroyAllWindows()

print("Video processing complete. Output saved to:", output_path)  # Modify if not saving extracted lip region


Video processing complete. Output saved to: C:\Users\Dhanush N\Desktop\Mini project\Dataset\outputs\Extractions\Demo\test.mp4


# Lip Comparison

In [3]:
#working Program 
#same above program with slow rate of video
import cv2

def compare_lip_movements(video_path1, video_path2, slow_factor=2):
  """
  Compares two videos side-by-side and displays them in a slow-motion manner.

  Args:
      video_path1: Path to the first video file.
      video_path2: Path to the second video file.
      slow_factor: An integer (default: 2) specifying how much to slow down the playback. Higher values result in slower playback.
  """

  # Open video files
  cap1 = cv2.VideoCapture(video_path1)
  cap2 = cv2.VideoCapture(video_path2)

  while True:
    # Read frames from the videos
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    # Break the loop if the videos end
    if not ret1 or not ret2:
      break

    # Display the frames side by side
    comparison_frame = cv2.hconcat([frame1, frame2])

    # Show the frame for a longer duration to slow down playback
    cv2.imshow('Lip Movements Comparison (Slow Motion)', comparison_frame)
    cv2.waitKey(int(1000 / slow_factor))  # Wait for specified milliseconds based on slow_factor

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

  # Release the video capture objects
  cap1.release()
  cap2.release()

  # Destroy all OpenCV windows
  cv2.destroyAllWindows()

# Example usage with slow motion effect
video_path1 = r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\outputs\Extractions\Demo\hello_2.mp4"
video_path2 = r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\outputs\Extractions\Demo\hello_4.mp4"
compare_lip_movements(video_path1, video_path2, slow_factor=4)  # Adjust slow_factor as needed


# Feedback

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def calculate_ssim(video_path1, video_path2):
    # Open video files
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    # Initialize SSIM values
    ssim_values = []

    while True:
        # Read frames from the videos
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        # Break the loop if the videos end
        if not ret1 or not ret2:
            break

        # Convert frames to grayscale
        gray_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Calculate SSIM between frames
        score, _ = ssim(gray_frame1, gray_frame2, full=True)
        ssim_values.append(score)

    # Release the video capture objects
    cap1.release()
    cap2.release()
    
    

    # Calculate average SSIM value
    average_ssim = np.mean(ssim_values)
    
    # Provide feedback based on the average SSIM
    if average_ssim > 0.8:
        print("The lip movements are very similar.")
    elif average_ssim > 0.6:
        print("The lip movements are moderately similar.")
    else:
        print("The lip movements are significantly different.")
    
# Example usage
video_path1 = r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\outputs\Extractions\Demo\hello_2.mp4"
video_path2 = r"C:\Users\Dhanush N\Desktop\Mini project\Dataset\outputs\Extractions\Demo\hello_4.mp4"
calculate_ssim(video_path1, video_path2)